In [1]:
# 라이브러리 로딩

from pandas_ods_reader import read_ods
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as W
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import argparse
import getpass
import json
import pandas as pd
import pprint
import sys
import time

In [2]:
wait_time_out = 10
driver = webdriver.Chrome()
wait = W(driver, wait_time_out)
driver.maximize_window()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 94
Current browser version is 96.0.4664.45 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00D9BDE3+2473443]
	Ordinal0 [0x00D36661+2057825]
	Ordinal0 [0x00C42438+1057848]
	Ordinal0 [0x00C6054A+1181002]
	Ordinal0 [0x00C5C435+1164341]
	Ordinal0 [0x00C59CAF+1154223]
	Ordinal0 [0x00C89ADF+1350367]
	Ordinal0 [0x00C8974A+1349450]
	Ordinal0 [0x00C855AB+1332651]
	Ordinal0 [0x00C62104+1188100]
	Ordinal0 [0x00C62F59+1191769]
	GetHandleVerifier [0x00F22266+1549718]
	GetHandleVerifier [0x00FCD4A7+2250711]
	GetHandleVerifier [0x00E2718B+521403]
	GetHandleVerifier [0x00E26229+517465]
	Ordinal0 [0x00D3B79D+2078621]
	Ordinal0 [0x00D3FB58+2095960]
	Ordinal0 [0x00D3FC92+2096274]
	Ordinal0 [0x00D49541+2135361]
	BaseThreadInitThunk [0x7571FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77677C7E+286]
	RtlGetAppContainerNamedObjectPath 

### Login to Autoway

In [3]:
# 로그인
def login(driver, userid, passwd, otp):
            
    # 카테고리 (서비스) 선택
    driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div[2]/div[3]/div/div[5]/label/div[1]/i').click()
    
    # 회사 선택
    driver.find_element_by_xpath('//*[@id="drpGroupCopList"]').click()

    value = "현대오토에버"
    select = Select(driver.find_element_by_id('drpGroupCopList'))
    select.select_by_visible_text(value)

    # UserID 입력
    username = driver.find_element_by_id("txtLoginID_Visible")
    username.clear()
    username.send_keys(userid)

    # Password 입력
    password = driver.find_element_by_id("password")
    password.clear()
    password.send_keys(passwd)

    # OTP 입력
    username = driver.find_element_by_id("code")
    username.clear()
    username.send_keys(otp)

    # Login 버튼 클릭
    login = driver.find_element_by_xpath('//*[@id="Login"]').click()

    return login

# ifrMain 으로 이동
def gobacktoMain(driver):
    driver.switch_to.default_content()
    iframe = driver.find_element_by_xpath('//iframe[@id="ifrMain"]')
    driver.switch_to.frame(iframe)

# ifrGnbMenu iframe 으로 이동
def clickTools(driver):
    driver.find_element_by_xpath('//*[@id="btnGnbMenu"]/i').click()
    time.sleep(0.5)
    iframe = driver.find_element_by_xpath('//iframe[@id="ifrGnbMenu"]')
    driver.switch_to.frame(iframe)
    driver.find_element_by_xpath('//*[@id="gnb_perfect_scroll"]/div[3]/a').click()
    time.sleep(0.5)
    
# i-sams 문자열로 app 찾기    
def searchiSams(driver):
    # 검색어 입력
    search = driver.find_element_by_id("ctl00_ContentPlaceHolder1_txtSearchText")
    search.clear()
    search.send_keys('i-sams')
    search.send_keys(Keys.ENTER)
    time.sleep(0.5)
    
# i-sams 팝업창 띄우기
def clickLinkofiSams(driver):
    ret = driver.find_element_by_xpath('//*[@id="aspnetForm"]/div[5]/div/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[2]/td[2]/a[1]').click()
    print(ret)
    time.sleep(0.5)

# i-sams 팝업창에서 링크 찾아 이동하기
def gotoPopUp_and_movetoiSams(driver):
    iframe = driver.find_element_by_xpath('//iframe[@name="InfoPopUpFrame"]')
    driver.switch_to.frame(iframe)
    # html = driver.page_source
    # html
    ret = driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div/div[1]/div[2]/table/tbody/tr[6]/td/div/a').click()
    print(ret)
    time.sleep(0.5)
    
# 진단실행 하기
def execute(driver, index, hostname):
    try:               
        # 진단 실행 대상 선택 (체크박스)
        time.sleep(3)
        checkboxes = driver.find_elements_by_class_name('asset-td-checkbox')
        total_rows_length = len(checkboxes)

        if total_rows_length > 0:
            for i in range(0, total_rows_length):
                checkboxes[i].click()
            time.sleep(1)
            
            try:
                # 진단 실행 버튼 Click
                execute = driver.find_element_by_id("bt-auditstart")
                execute.click()
                time.sleep(1)

                try:
                    # 진단 실행 Popup 창 및 경고 창 확인
                    execute = driver.find_element_by_id("bt-modal-startaudit-ok")
                    execute.click()
                    time.sleep(1)
                finally:
                    alert_obj = driver.switch_to.alert
                    # print(alert_obj)
                    alert_obj.accept()
                    # print(driver.window_handles)
                    print("Test requested : ", index, hostname)
                    time.sleep(1)
            except:
                pass
        else:
            print("Hostname doesn't exists : ", index, hostname)
    finally:
        # 자산 탭 버튼 Click
        execute = driver.find_element_by_id("tree-tab-asset")
        execute.click()
        time.sleep(1)
        
# 진단실행 취소 하기
def cancelExecute(driver, index, hostname):

    try:
        # 진단 실행 Popup 창 취소 버튼
        execute = driver.find_element_by_xpath('//*[@id="modal-startaudit"]/div[1]/table/tbody[2]/tr/td/div/button[2]')
        execute.click()
        print("Test cancelled : ", index, hostname)
        time.sleep(1)

    finally:
        # 자산 탭 버튼 Click
        execute = driver.find_element_by_id("tree-tab-asset")
        execute.click()
        time.sleep(1)

In [4]:
# 카테고리 (서비스) 선택
while(1):
    try:
        driver.get('https://autoway.hyundai.net/login')
        element = W(driver, 10).until(E.presenence_of_element_located((By.ID, "drpGroupCopList")))
    finally:
        break

In [5]:
# Login 정보 입력
# userid = config.id
# passwd = config.pwd
# otp = config.otp
# if otp == None:
#     otp = input("OTP     :")
#     print(otp)        

userid = '9368265'
passwd = 'TldhTl@123'
otp = '65421472'

ret = login(driver, userid, passwd, otp)
time.sleep(5)
print(ret)

None


In [6]:
print(driver.current_window_handle)

CDwindow-C5A7EEC42DBF2886CF77451C37F3019A


### Login to i-Sams

In [7]:
print(driver.window_handles)

['CDwindow-C5A7EEC42DBF2886CF77451C37F3019A']


In [8]:
driver.switch_to.window(driver.window_handles[0])

In [9]:
# 기본 iframe 으로 이동
driver.switch_to.default_content()

# ifrGnbMenu iframe 으로 이동후, 도구 메뉴 클릭
clickTools(driver)

# ifrMain 으로 이동
gobacktoMain(driver)

# i-sams 문자열로 app 찾기  
searchiSams(driver)

# i-sams 팝업창 띄우기
clickLinkofiSams(driver)

# i-sams 팝업창에서 링크 찾아 이동하기
gotoPopUp_and_movetoiSams(driver)
time.sleep(15)

None
None


In [10]:
print(driver.current_window_handle)

CDwindow-C5A7EEC42DBF2886CF77451C37F3019A


In [11]:
print(driver.window_handles)

['CDwindow-C5A7EEC42DBF2886CF77451C37F3019A', 'CDwindow-3F9424AE4B78E821AF37AE8600BB83E4']


In [12]:
driver.switch_to.window(driver.window_handles[1])

In [13]:
print(driver.current_window_handle)

CDwindow-3F9424AE4B78E821AF37AE8600BB83E4


In [14]:
print(driver.window_handles)

['CDwindow-C5A7EEC42DBF2886CF77451C37F3019A', 'CDwindow-3F9424AE4B78E821AF37AE8600BB83E4']


### Data Loading

In [3]:
base_path = r"./21년도 정보시스템 및 인프라 정기점검_11월02일기준_모니터링(전체).ods"
sheet = 'Sheet'

df = read_ods(base_path, sheet, headers=True)

df.head()

,점검상태,점검구분,점검 대상 코드(CI번호),자산 종류,IP,HOSTNAME,담당팀명,담당자명,점검상태.1,점검완료일,점검자명,승인완료일,승인자명
0,인프라 점검,DBMS,AES-S-SO-13-0191,SQL Server,10.10.97.130,VMRDEV16_MSSQLSERVER,서비스시스템팀,노기훈,점검완료,2021-03-29T00:00:00,노기훈,2021-03-29T00:00:00,탁상조
1,인프라 점검,DBMS,AES-S-SO-17-0041,Oracle DBMS,10.10.161.133,kmbdbp02_KMCMB2,미들웨어기술팀,최지수,점검완료,2021-07-21T00:00:00,setAutoInspectionTarget,None,None
2,인프라 점검,DBMS,AES-S-SO-17-0042,Oracle DBMS,10.10.161.132,kmbdbp01_KMCMB1,미들웨어기술팀,최지수,점검완료,2021-07-21T00:00:00,setAutoInspectionTarget,None,None
3,인프라 점검,DBMS,H10-S-SO-13-0110,Oracle DBMS,10.10.195.24,corddd00_UD2,차세대ERP추진TFT,변성현,점검완료,2021-06-23T00:00:00,변성현,2021-07-19T00:00:00,남호열
4,인프라 점검,DBMS,H10-S-SO-13-0111,Oracle DBMS,10.10.195.21,cordpd00_up3,차세대ERP추진TFT,변성현,점검완료,2021-06-15T00:00:00,변성현,2021-07-19T00:00:00,남호열


In [4]:
df = df[(df["담당팀명"] == '클라우드운영센터') & (df["점검상태.1"] == "미점검")]

df.describe()

,점검상태,점검구분,점검 대상 코드(CI번호),자산 종류,IP,HOSTNAME,담당팀명,담당자명,점검상태.1,점검완료일,점검자명,승인완료일,승인자명
count,383,383,383,383,383,383,383,383,383,88,88,0,0
unique,1,2,383,3,383,383,1,4,1,18,4,0,0
top,인프라 점검,OS,HKM-S-SO-20-5407,CENTOS,10.11.52.235,ccskrccspdsrediss01.krcloud_redisDB,클라우드운영센터,장재봉,미점검,2018-08-09T00:00:00,김정환,NaN,NaN
freq,383,377,1,309,1,1,383,370,383,27,69,NaN,NaN


In [5]:
df = df[['HOSTNAME', '담당팀명', '담당자명', '점검상태.1']]
df = df.set_axis(["hostname", "team", "owner", "status"], axis=1)

df.describe()

,hostname,team,owner,status
count,383,383,383,383
unique,383,1,4,1
top,ccskrccspdsrediss01.krcloud_redisDB,클라우드운영센터,장재봉,미점검
freq,1,383,370,383


In [6]:
df

,hostname,team,owner,status
1071,ccskrccspdsrediss01.krcloud_redisDB,클라우드운영센터,김의현,미점검
1072,ccskrccspdsrediss03.krcloud_redisDB,클라우드운영센터,김의현,미점검
1077,ccskrccspdsredisp02.krtms_redisDB,클라우드운영센터,김의현,미점검
1078,ccskrccspdsredisp03.krtms_redisDB,클라우드운영센터,김의현,미점검
1082,ccskrccspdsredisp01.krtms_redisDB,클라우드운영센터,김의현,미점검
...,...,...,...,...
8571,krcloudstgcomdr311.krcloud,클라우드운영센터,장재봉,미점검
8575,krcloudstgcomdr312.krcloud,클라우드운영센터,장재봉,미점검
8577,krcloudstgcomdr310.krcloud,클라우드운영센터,장재봉,미점검
8578,krcloudstgcomdr307.krcloud,클라우드운영센터,장재봉,미점검


In [7]:
df.reset_index(inplace=True)

In [8]:
df

,index,hostname,team,owner,status
0,1071,ccskrccspdsrediss01.krcloud_redisDB,클라우드운영센터,김의현,미점검
1,1072,ccskrccspdsrediss03.krcloud_redisDB,클라우드운영센터,김의현,미점검
2,1077,ccskrccspdsredisp02.krtms_redisDB,클라우드운영센터,김의현,미점검
3,1078,ccskrccspdsredisp03.krtms_redisDB,클라우드운영센터,김의현,미점검
4,1082,ccskrccspdsredisp01.krtms_redisDB,클라우드운영센터,김의현,미점검
...,...,...,...,...,...
378,8571,krcloudstgcomdr311.krcloud,클라우드운영센터,장재봉,미점검
379,8575,krcloudstgcomdr312.krcloud,클라우드운영센터,장재봉,미점검
380,8577,krcloudstgcomdr310.krcloud,클라우드운영센터,장재봉,미점검
381,8578,krcloudstgcomdr307.krcloud,클라우드운영센터,장재봉,미점검


In [36]:
count = 0
checkRequest = 0
pass_index = -1
timeout = 5

for index, row in df.iterrows():
    try:
        hostname = row['hostname'].split(".")[0]+"."+row['hostname'].split(".")[1]
    except:
        hostname = row['hostname'].split(".")[0]
        
    if index < pass_index:
        pass
    else:
        try:
            search = driver.find_element_by_id("input-text-list-asset-search")
            time.sleep(3)
            search.clear()
            search.click()
            search.send_keys(hostname)
            search.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # Check if new DOM is loaded
#             try:
#                 element_present = W(driver, 10).until(E.presence_of_element_located((By.ID, "sysinfo-choice-temp-name")))
#                 W(driver, timeout).until(element_present)
#             except TimeoutException:
#                 print ("Timed out waiting for page to load")
#                 print(element)
#             break
            
            # 진단 실행 대상 선택 (체크박스)
            checkboxes = driver.find_elements_by_class_name('asset-td-checkbox')
            total_rows_length = len(checkboxes)

            if total_rows_length > 0:
                for i in range(0, total_rows_length):
                    checkboxes[i].click()
                time.sleep(1)

                execute(driver, index, hostname)
                checkRequest = checkRequest + 1
            else:
                hostname = hostname.split(".")[0]
                
                try:
                    search = driver.find_element_by_id("input-text-list-asset-search")
                    time.sleep(3)
                    search.clear()
                    search.click()
                    search.send_keys(hostname)
                    search.send_keys(Keys.ENTER)
                    time.sleep(3)

                    # 진단 실행 대상 선택 (체크박스)
                    checkboxes = driver.find_elements_by_class_name('asset-td-checkbox')
                    total_rows_length = len(checkboxes)

                    if total_rows_length > 0:
                        for i in range(0, total_rows_length):
                            checkboxes[i].click()
                        time.sleep(1)

                        execute(driver, index, hostname)
                        checkRequest = checkRequest + 1
                    else:
                        print("Hostname doesn't exist : ", index, hostname)                        
                except:
                    print("Test not requested : ", index, hostname)
        except:
            cancelExecute(driver, index, hostname)

Test requested :  341 krcloudstgcomdr208.krcloud
Hostname doesn't exist :  342 krcloudprdcomdr1310
Hostname doesn't exist :  343 krcloudprdcomdr1203
Hostname doesn't exist :  344 krcloudprdcomdr1204
Test requested :  345 krcloudstgcomdr202.krcloud
Hostname doesn't exist :  346 krcloudprdcomdr1307
Hostname doesn't exist :  347 krcloudprdcomdr1305
Hostname doesn't exist :  348 krcloudprdcomdr1202
Test requested :  349 krcloudstgcomdr203.krcloud
Hostname doesn't exist :  350 krcloudprdcomdr1207
Hostname doesn't exist :  351 krcloudprdcomdr1306
Hostname doesn't exist :  352 krcloudprdcomdr1301
Hostname doesn't exist :  353 krcloudprdcomdr1304
Hostname doesn't exist :  354 krcloudprdcomdr1201
Test requested :  355 krcloudstgcomdr211.krcloud
Hostname doesn't exist :  356 krcloudprdcomdr1208
Test requested :  357 krcloudstgcomdr201.krcloud
Test requested :  358 krcloudstgcomdr212.krcloud
Hostname doesn't exist :  359 krcloudprdcomdr1209
Test requested :  360 krcloudstgcomdr204.krcloud
Hostnam

## 진단실행 여부 (1ST RUN, 2021.11.11)

In [32]:
print("총 갱신 레코드 / 요청 작업 수 / 갱신 성공(작업 수): ========== %d / %d / %d" % (total, checkRequest, count) )

총 갱신 레코드 / 요청 작업 수 / 갱신 성공(작업 수): ========== 383 / 0 / 0


In [22]:
# html = driver.page_source
# html

In [ ]:
# 이슈 있는 호스트
# 83 ~ 86

In [ ]:
# %load isams.py
# (env) D:\rpa>python isams.py --ods_fn '2021년 i-sams 조치 대상 리스트_0902.ods' --owner 장재봉 --id 9306532 --pwd Tmvkzm1!